In [1]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [2]:
ls

README.md          jupyter_notebooks/ run.py
donkey_golf/       requirements.txt


In [3]:
from inspect import getsource

In [4]:
import donkey_golf as dg

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [5]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [6]:
vamp = dg.data_things.LoadLeaderboard()

2019-05-08 23:00:06,882 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-08 23:00:06,897 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 23:00:07,164 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056551
2019-05-08 23:00:07,165 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [ ]:
vamp.tourney_status

In [7]:
new = dg.data_things.PullLeaderboard(user_id=10)

2019-05-08 23:00:09,702 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-08 23:00:09,719 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 23:00:10,029 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056551
2019-05-08 23:00:10,030 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [8]:
new.tourney_status

'pre_tourney'

In [9]:
new.run()

2019-05-08 23:00:10,327 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-08 23:00:10,590 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users pre_tourney team
2019-05-08 23:00:10,591 - donkey_golf.data_things.base_class - INFO - Executing run_sql


ValueError: cannot convert float NaN to integer

In [ ]:
new.pull_tourney_leaderboard()

In [11]:
new.user_df

,id,tourney_id,player
0,10,401056551,Brooks Koepka
1,10,401056551,Patrick Reed
2,10,401056551,Marc Leishman
3,10,401056551,Ryan Moore
4,10,401056551,Ryan Palmer
5,10,401056551,Brian Harman
6,10,401056551,Brooks Koepka
7,10,401056551,Marc Leishman
8,10,401056551,Branden Grace
9,10,401056551,Charles Howell III


In [ ]:
new.user_id

In [ ]:
new.pull_users_team()

In [13]:
new.data

,pos,to_par,r1,r2,r3,r4,tot,earnings,fedex_pts,tourney_id,donkey_score,missed_cut,player_left,team_count,load_date,player,on_team
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooks Koepka,Yes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooks Koepka,Yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Patrick Reed,Yes
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marc Leishman,Yes
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marc Leishman,Yes
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ryan Moore,Yes
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ryan Palmer,Yes
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brian Harman,Yes
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Branden Grace,Yes
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charles Howell III,Yes


In [ ]:
new.pull_tourney_leaderboard()

In [12]:
getattr(new, 'user_df')

,id,tourney_id,player
0,10,401056551,Brooks Koepka
1,10,401056551,Patrick Reed
2,10,401056551,Marc Leishman
3,10,401056551,Ryan Moore
4,10,401056551,Ryan Palmer
5,10,401056551,Brian Harman
6,10,401056551,Brooks Koepka
7,10,401056551,Marc Leishman
8,10,401056551,Branden Grace
9,10,401056551,Charles Howell III


In [ ]:
print(getsource(new.pull_tourney_leaderboard))

In [ ]:
new.user_df

In [ ]:
pd.merge(new.data, new.user_df,
        how='outer',
        on='player')

In [ ]:
new.tourney_status

In [ ]:
gs = dg.data_things.GameScoreboard()

In [ ]:
gs.tourney_status

In [ ]:
gs.run()

In [ ]:
wha = dg.data_things.LoadWorldRankings()

In [ ]:
wha = dg.data_things.TournamentInfo()

In [ ]:
wha.run()

In [ ]:
draft = dg.data_things.DraftDay()

In [ ]:
draft.run()

In [ ]:
df = draft.data

In [ ]:
for col in ['current_rank','events_played','rank']:
    self.data.loc[self.data[col].notnull(), col] = self.data.loc[self.data[col].notnull(), col].astype(int).astype(str)

In [ ]:
df

In [ ]:
draft.data['current_rank']

In [ ]:
vamp.new_determine_status()['value'][0].strip()

In [ ]:
vamp.run()

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.load_status_sql

In [ ]:
with psycopg2.connect(vamp.conf.db_url) as conn:
    cur = conn.cursor()
    cur.execute("delete from donkey_variables where metric = 'current_tourney_status';")
    cur.execute("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")
    #cur.close()

In [ ]:
conn = psycopg2.connect(host=pg_credential.hostname,
                        port=pg_credential.port,
                        user=pg_credential.username,
                        password=pg_credential.password,
                        database=pg_credential.path[1:]) # To remove slash

cursor = conn.cursor()
cursor.execute("INSERT INTO a_table (c1, c2, c3) VALUES(%s, %s, %s)", (v1, v2, v3))

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.tourney_status

In [ ]:
pd.read_sql("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.run()

In [ ]:
cmon.tourney_status

In [ ]:
cmon.pull_tourney_leaderboard()

In [ ]:
if cmon.data.empty:
    print("EMPTY")

In [ ]:
cmon.tourney_status

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
getattr(cmon, 'tourney_statusss', 'whoops')

In [ ]:
sb = dg.data_things.GameScoreboard()

In [ ]:
sb.run()

In [ ]:
vamp.golfer_count_by_team

In [ ]:
df = vamp.data

In [ ]:
df.head(2)

In [ ]:
df.loc[df['team_count']> 0]

In [ ]:
huh = dg.data_things.PullLeaderboard(user_id=6)

In [ ]:
huh.run()

In [ ]:
huh.data['team_count'].value_counts()

In [ ]:
df_tourney = huh.data
df_tourney = df_tourney.loc[df_tourney['team_count'] > 0]

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
df_tourney

In [ ]:
ada = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
ada.pull_tourney_leaderboard()

In [ ]:
ada.data.team_count.value_counts()

In [ ]:
ada.calc_refresh_date()

In [ ]:
ada.determine_tourney_status()

In [ ]:
ada.pull_users_team()

In [ ]:
ada.user_df

In [ ]:
ada.sql_user